In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request, urlopen
import requests

In [2]:
headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
request_soup = requests.get('https://pubs.acs.org/', headers=headers)
soup = BeautifulSoup(request_soup.text, 'html.parser')

In [3]:
journals_soup = soup.findAll("div", {"class": "quick-search"})

In [4]:
journals = soup.findAll("option")

In [5]:
#def get_link(journal_code):
#    array = []
#    for i in journal_code:
#        link = "https://pubs.acs.org/journal/{}".format(i)
#        array.append(link)
#    return (array)

In [6]:
#links = get_link(journal_code)
#links_temp = get_link(journal_code_temp)
#links_temp

## Journals

In [7]:
#headers = ({'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
#req_impact = requests.get('https://pubs.acs.org/journal/achre4', headers=headers)
#soup_impact = BeautifulSoup(req_impact.text, 'html.parser')

In [8]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
#from fake_useragent import UserAgent
import time

In [18]:
journal_name = []
journal_code = []
journal_code_temp = []
current_vol = []
current_issue = []
current_date = []
journal_impact = []
journal_citation = []


In [39]:
options = Options()   
browser = webdriver.Firefox()
def main():

    for i in journal_code_temp:

        url = 'https://pubs.acs.org/journal/'+str(i)
        browser.get(url)
        acs_scraper(journals, browser)        
    

In [37]:
def acs_scraper(journals, browser):   
    journal_name = []
    journal_code = []
    journal_code_temp = []
    current_vol = []
    current_issue = []
    current_date = []
    journal_impact = []
    journal_citation = []
    
    time.sleep(10)
    journal_name = [s.text.strip() for s in journals]
    journal_name_temp = journal_name[1:2]
    journal_name_temp
    
    journal_code = [s['value'] for s in journals]
    journal_code_temp = journal_code[1:2]
    journal_code_temp
    
    issue = browser.find_elements_by_xpath('.//div[@class = "jhHeader_meta"]')  
    print (issue)
    for i in issue:
        current_vol.append(i.find_element_by_xpath('.//span[1]').text)
        current_issue.append(i.find_element_by_xpath('.//span[2]').text)
        current_date.append(i.find_element_by_xpath('.//p').text)
        
    journal_impact.append(browser.find_element_by_class_name('jhHeader_impact').text)
    journal_citation.append(browser.find_element_by_class_name('jhHeader_citations').text)
    df = pd.DataFrame({"name":journal_name,"code":journal_code,"impact":journal_impact,"citation" : journal_citation, "current volume" : current_vol, "current issue" : current_issue, "current date" : current_date})
    df.to_csv("acs_journals.csv")
    #return journal_name, journal_code, current_vol, current_issue, current_date, journal_impact, journal_citation

In [35]:
if __name__ == '__main__':
    #acs_scraper(journals, browser, journal_name, journal_code, journal_code_temp, current_vol, current_issue, current_date, journal_impact, journal_citation)  
    main()

In [38]:
acs_scraper(journals, browser)

[]


NoSuchElementException: Message: Unable to locate element: .jhHeader_impact
